In [2]:
#GeoMAPA por provincia: Renta promedio

import folium
import pandas as pd
import re

In [3]:
# Función TOCHA TOCHA
bbdd = ["Alava","Albacete","Alicante","Almeria","Asturias","Avila","Badajoz","Barcelona",
        "Bizkaia","Burgos","Caceres","Cadiz","Cantabria","Islas Baleares","Castellon","Ceuta","Ciudad Real",
        "Cordoba","Coruña","Cuenca","Gipuzkoa","Girona","Granada","Guadalajara","Huelva","Huesca","Jaen","La Rioja",
        "Las Palmas","Leon","LLeida","Lugo","Madrid","Malaga","Melilla","Murcia","Navarra","Ourense","Palencia","Pontevedra",
       "Salamanca","Santa Cruz de Tenerife","Segovia","Sevilla","Soria","Tarragona","Teruel","Toledo","Valencia",
        "Valladolid","Zamora","Zaragoza"]
def get_BBDD_rentas (bbdd):
    #Carga de ficheros CSV
    base = []
    NIE = []
    renta = []
    poblacion = []
    provincias = []
    for i in bbdd:
        provincia = i
        datos = pd.read_csv('datos_renta/'+i+'.csv', index_col=False,encoding='iso-8859-1'
                            ,sep=';',skiprows=5, decimal= ',',thousands='.')
        datos.rename(columns={'Unnamed: 0':'identificador','2016':'renta'},inplace=True)
        datos.drop(datos.tail(3).index,inplace=True)
        for i in range(len(datos.identificador)):
            provincias.append(provincia)
        for i in range(len(datos.identificador)):
            NIE_value = int(re.findall("[0-9]+", datos.identificador[i])[0])
            NIE.append(NIE_value) 
        for i in range(len(datos.identificador)):
             nombre = re.findall("[a-zA-Záéíóúñ\\- /]+", datos.identificador[i])[0]
             nombre = re.sub(" sección","",nombre)
             nombre = re.sub(" secci","",nombre)
             poblacion.append(nombre)
        for i in range(len(datos.identificador)):
            if datos.renta[i] == '.':
                datos.renta[i] = float('NaN')
            renta_value = float(datos.renta[i])
            renta.append(renta_value)
    d = {'provincia':provincias,'poblacion':poblacion,'NIE':NIE,'renta':renta}
    datos = pd.DataFrame(d)
    return datos

basedatos = get_BBDD_rentas(bbdd)
basedatos

,provincia,poblacion,NIE,renta
0,Alava,Agurain/Salvatierra,1051,12374.0
1,Alava,Alegría-Dulantzi,1001,13086.0
2,Alava,Amurrio,1002,13691.0
3,Alava,Añana,1049,15548.0
4,Alava,Aramaio,1003,17194.0
...,...,...,...,...
37016,Zaragoza,Zaragoza,5029712010,14085.0
37017,Zaragoza,Zuera,5029801001,12307.0
37018,Zaragoza,Zuera,5029801002,10848.0
37019,Zaragoza,Zuera,5029801003,10726.0


In [67]:
datos_renta = basedatos.groupby('provincia').renta.mean()
datos_renta = datos_renta.rename(columns={"provincia": "renta"})
datos_renta.head()

provincia
Alava       13.891549
Albacete     9.450120
Alicante     8.919547
Almeria      8.494320
Asturias    12.146515
dtype: float64

In [84]:
geomap = pd.read_json('provincias_espanolas.geojson')

0     {'type': 'Feature', 'geometry': {'type': 'Poly...
1     {'type': 'Feature', 'geometry': {'type': 'Poly...
2     {'type': 'Feature', 'geometry': {'type': 'Mult...
3     {'type': 'Feature', 'geometry': {'type': 'Poly...
4     {'type': 'Feature', 'geometry': {'type': 'Poly...
5     {'type': 'Feature', 'geometry': {'type': 'Mult...
6     {'type': 'Feature', 'geometry': {'type': 'Poly...
7     {'type': 'Feature', 'geometry': {'type': 'Poly...
8     {'type': 'Feature', 'geometry': {'type': 'Poly...
9     {'type': 'Feature', 'geometry': {'type': 'Poly...
10    {'type': 'Feature', 'geometry': {'type': 'Mult...
11    {'type': 'Feature', 'geometry': {'type': 'Mult...
12    {'type': 'Feature', 'geometry': {'type': 'Mult...
13    {'type': 'Feature', 'geometry': {'type': 'Poly...
14    {'type': 'Feature', 'geometry': {'type': 'Mult...
15    {'type': 'Feature', 'geometry': {'type': 'Mult...
16    {'type': 'Feature', 'geometry': {'type': 'Poly...
17    {'type': 'Feature', 'geometry': {'type': '

In [4]:
# Incializamos el mapa
m = folium.Map(location=[40.42, -3.7], zoom_start=6, tiles='cartodbpositron')
bins = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]
# Add the color for the chloropleth:
folium.Choropleth(
    geo_data='provincias_espanolas.geojson',
    data=basedatos,
    columns=['provincia', 'renta'],
    key_on='feature.properties.provincia',
    fill_color='YlGn',
    bins=bins
).add_to(m)
m

KeyError: 'YlGn'